In [3]:
from agentscope.agent import ReActAgent, UserAgent
from agentscope.formatter import DashScopeChatFormatter
from agentscope.memory import InMemoryMemory
from agentscope.model import DashScopeChatModel
from agentscope.tool import (
    Toolkit,
    execute_shell_command,
    execute_python_code,
    view_text_file,
)

from agentscope.message import (
    Msg,
    ToolUseBlock,
    ToolResultBlock,
    TextBlock,
    AudioBlock,
)

## 记忆模块
InMemoryMemory

In [4]:
print(InMemoryMemory())

In [20]:
memory = InMemoryMemory()
await memory.add(Msg(role="user", content="Hello, how are you?", name="user"))
await memory.add(Msg(role="assistant", content="I'm fine, thank you!", name="assistant"))
print(await memory.get_memory())

[Msg(id='Q3ZwnPJHDufMpJhbSfwHYb', name='user', content='Hello, how are you?', role='user', metadata=None, timestamp='2026-01-09 08:02:56.901', invocation_id='None'), Msg(id='SdZhZxCVeQs5D6XSJM2cPb', name='assistant', content="I'm fine, thank you!", role='assistant', metadata=None, timestamp='2026-01-09 08:02:56.901', invocation_id='None')]


In [8]:
# 一条短消息
message = Msg(role="user", content="Hello, how are you?", name="user")



In [10]:
print(message)

Msg(id='WY4GJpiKUZsLUfR7VornoM', name='user', content='Hello, how are you?', role='user', metadata=None, timestamp='2026-01-09 07:49:47.588', invocation_id='None')


In [15]:
print(memory.content)

[]


In [ ]:
memory = InMemoryMemory()

await memory.add(Msg(name="1", role="user", content="你好"))
await memory.add(Msg(name="2", role="assistant", content="こんにちは！"))

print(await memory.size())          # 输出: 2
print(await memory.get_memory())    # 输出: [Msg1, Msg2]


2
[Msg(id='aEVxX9aLJw84japJB9sweY', name='1', content='你好', role='user', metadata=None, timestamp='2026-01-09 08:08:50.902', invocation_id='None'), Msg(id='iK8QcowHEaxKSvcrcu4wd6', name='2', content='こんにちは！', role='assistant', metadata=None, timestamp='2026-01-09 08:08:50.902', invocation_id='None')]
2


In [23]:

# 保存状态
state = memory.state_dict()
print(state)


{'content': [{'id': 'aEVxX9aLJw84japJB9sweY', 'name': '1', 'role': 'user', 'content': '你好', 'metadata': None, 'timestamp': '2026-01-09 08:08:50.902'}, {'id': 'iK8QcowHEaxKSvcrcu4wd6', 'name': '2', 'role': 'assistant', 'content': 'こんにちは！', 'metadata': None, 'timestamp': '2026-01-09 08:08:50.902'}]}


In [ ]:

# 清空后恢复
await memory.clear()
memory.load_state_dict(state)
print(await memory.size())          # 再次输出: 2

## 模型调用

In [2]:
import asyncio
import os
from dashscope.aigc.generation import AioGeneration

# 请提前设置环境变量：export DASHSCOPE_API_KEY=sk-xxxxxxxxxxxx
# 或者直接在这里填写（不推荐）
DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY", "your-api-key-here")

async def non_stream_demo():
    """非流式调用：一次性返回完整结果（适合后台处理、需要完整文本的场景）"""
    print("\n=== 非流式调用演示 ===\n")
    print("用户：北京2026年天气怎么样？\n")
    print("Friday（一次性完整回复）：")

    payload = {
        "model": "qwen-max",                  # 可换成 qwen-plus、qwen-turbo 等
        "messages": [
            {"role": "system", "content": "You are a helpful assistant named Friday."},
            {"role": "user", "content": "北京2026年天气怎么样？"}
        ],
        "stream": False,                      # 关键：关闭流式
        "result_format": "message",
        "temperature": 0.7,                   # 控制创造性：0.0~1.0，越高越随机
        "top_p": 0.8,                         # 核采样，建议和 temperature 二选一调整
        "max_tokens": 512,                    # 最大输出 token 数，防止太长
    }

    response = await AioGeneration.call(
        api_key=DASHSCOPE_API_KEY,
        **payload
    )

    if response.status_code == 200:
        content = response.output.choices[0].message.content
        print(content)

        # 显示 token 使用情况
        usage = response.usage
        print(f"\n\nToken 使用：输入 {usage.input_tokens} + 输出 {usage.output_tokens} = 总计 {usage.total_tokens}")
    else:
        print(f"请求失败：{response.message}")

async def stream_demo():
    """流式调用：实时逐字/逐 token 返回（适合聊天界面“打字机”效果）"""
    print("\n\n=== 流式调用演示 ===\n")
    print("用户：北京2026年天气怎么样？\n")
    print("Friday（实时打字）：", end="", flush=True)

    payload = {
        "model": "qwen-max",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant named Friday."},
            {"role": "user", "content": "北京2026年天气怎么样？"}
        ],
        "stream": True,                       # 关键：开启流式
        "incremental_output": True,           # 推荐：只返回增量内容（更自然的打字效果）
        "result_format": "message",
        "temperature": 0.85,                  # 流式时可以稍微提高，回复更生动
        "max_tokens": 512,
    }

    response = await AioGeneration.call(
        api_key=DASHSCOPE_API_KEY,
        **payload
    )

    full_content = ""
    async for chunk in response:
        if chunk.status_code != 200:
            print(f"\n\n流式错误：{chunk.message}")
            return

        if chunk.output and chunk.output.choices:
            delta = chunk.output.choices[0].message.content
            if delta:
                print(delta, end="", flush=True)
                full_content += delta

    print("\n\n（流式输出结束）")
    # 流式结束后，chunk 中也会带最终的 usage
    if hasattr(response, "usage"):
        usage = response.usage
        print(f"Token 使用：输入 {usage.input_tokens} + 输出 {usage.output_tokens} = 总计 {usage.total_tokens}")

async def main():
    # 依次运行两个 demo，让你直观对比区别
    await non_stream_demo()
    await stream_demo()

    print("\n\n=== 总结对比 ===")
    print("• 非流式（stream=False）：")
    print("  - 优点：简单，一次性拿到完整结果，便于后续处理（如总结、存储）")
    print("  - 缺点：用户需要等待整个回复生成完毕，体验上有延迟")
    print("  - 适合：后台任务、批量生成、需要完整文本分析的场景")
    print("")
    print("• 流式（stream=True + incremental_output=True）：")
    print("  - 优点：实时显示“打字”效果，用户感知响应更快，体验更好")
    print("  - 缺点：代码稍复杂，需要 async for 处理")
    print("  - 适合：聊天界面、WebSocket、实时交互应用")
    print("")
    print("常用参数建议：")
    print("• temperature：0.3~0.5（更确定、严谨） vs 0.7~0.9（更有创意）")
    print("• top_p：通常设 0.8~0.95，与 temperature 互补")
    print("• max_tokens：根据上下文窗口控制，避免超限（qwen-max 支持最大 ~32k tokens）")

if __name__ == "__main__":
    await main()


=== 非流式调用演示 ===

用户：北京2026年天气怎么样？

Friday（一次性完整回复）：
目前无法提供2026年北京具体的天气预报。天气预报通常只能准确预测未来几天到一周左右的情况，而长期的气候预测则依赖于气候模型和历史数据，但这些预测也主要是趋势性的，并不能精确到具体某一天的天气状况。

如果您对北京未来几年的大致气候变化感兴趣，可以关注气象科研机构发布的气候研究报告或长期气候预测。这类信息可能会给出一些关于温度、降水等平均值变化的趋势性描述，但依然存在不确定性。对于更短期的天气情况，请参考当地气象部门发布的最新天气预报。


Token 使用：输入 30 + 输出 119 = 总计 149


=== 流式调用演示 ===

用户：北京2026年天气怎么样？

Friday（实时打字）：目前无法提供2026年的具体天气预报，因为长期天气预测技术尚不能准确到数年后的具体天气情况。通常，可靠的天气预报可以提前大约一周给出，而气候趋势的预测则可以在更长的时间范围内（如几个月或一个季节）进行一定程度上的估计。

如果您对北京未来几年可能面临的气候变化感兴趣，可以关注气象科学研究机构发布的关于全球及区域气候变化的趋势报告。这些报告虽然不能提供具体的天气状况，但能够帮助人们了解未来气候的大致走向。例如，根据一些研究，全球变暖可能会导致某些地区的极端天气事件变得更加频繁和强烈。不过，请注意实际天气情况会受到多种因素的影响，具体情况还需以官方气象部门发布的最新信息为准。

（流式输出结束）


=== 总结对比 ===
• 非流式（stream=False）：
  - 优点：简单，一次性拿到完整结果，便于后续处理（如总结、存储）
  - 缺点：用户需要等待整个回复生成完毕，体验上有延迟
  - 适合：后台任务、批量生成、需要完整文本分析的场景

• 流式（stream=True + incremental_output=True）：
  - 优点：实时显示“打字”效果，用户感知响应更快，体验更好
  - 缺点：代码稍复杂，需要 async for 处理
  - 适合：聊天界面、WebSocket、实时交互应用

常用参数建议：
• temperature：0.3~0.5（更确定、严谨） vs 0.7~0.9（更有创意）
• top_p：通常设 0.8~0.95，